In [66]:
import pandas as pd
from haversine import haversine
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors 
import numpy as np
import math
import copy
import itertools
import networkx as nx # tool for graphs and graph algorithms

In [67]:
data = pd.read_csv('nyc_nodes.csv')
dfn = pd.DataFrame(data)
# load links
data = pd.read_csv('nyc_links.csv')
dfl = pd.DataFrame(data)
# load population
data = pd.read_csv('nyc_population.csv')
dfp = pd.DataFrame(data)

#For Q1 part (a)
dfs = pd.DataFrame(columns = ["distance", "time"])
for index,row in dfl.iterrows():
    start_point = row['start']
    end_point = row['end']
    #time = row['delay5pm']
    start_point_coord_row = dfn[(dfn.name==start_point)].index[0]
    end_point_coord_row = dfn[(dfn.name==end_point)].index[0]
    start_point_coord = dfn.loc[start_point_coord_row,['lat','lon']].tolist()
    end_point_coord = dfn.loc[end_point_coord_row,['lat','lon']].tolist()
    dfs.loc[index] = [haversine(start_point_coord, end_point_coord)*1000,row['time']]

In [68]:
ambulance_dfl = copy.deepcopy(dfl)    #travel time for ambulance
ambulance_dfl['time'] = 0.9*dfs['time']

In [80]:
#For Q2 part (b)
total_population = sum(dfp['pop'].tolist())
call_prob = []
for index,row in dfp.iterrows():
    call_prob.append(row['pop']/total_population)
population_with_prob = copy.deepcopy(dfp) #a new dataframe with a new column showing the probability 
population_with_prob['prob'] = call_prob

#For Q2 part (d)
poi = dfp['name'].tolist()  #list of representaitve points
G = nx.from_pandas_edgelist(ambulance_dfl, 'start', 'end', 'time')
""" 
Please use this "distances" matrix for your case 1
"""
distances = np.zeros([len(poi),len(poi)])   #a matrix showing shortest time needed bewtween each pair of points
for i in range(len(poi)):
    out = nx.single_source_dijkstra(G, poi[i], weight='time')
    results = pd.DataFrame({'node_id':poi})
    outcome = results['node_id'].map(out[0]) 
    distances[i,:] = outcome

In [81]:
distances

array([[   0.        ,  426.51881335,  971.49925454, ...,  504.82638967,
         676.6086324 , 1473.34513619],
       [ 426.51881335,    0.        , 1138.54584839, ...,  246.20980309,
         257.84725604, 1736.19190112],
       [ 971.49925454, 1138.54584839,    0.        , ...,  915.26031027,
        1140.04829103,  682.41846061],
       ...,
       [ 504.82638967,  246.20980309,  915.26031027, ...,    0.        ,
         337.10801828, 1550.46648764],
       [ 676.6086324 ,  257.84725604, 1140.04829103, ...,  337.10801828,
           0.        , 1775.25446841],
       [1473.34513619, 1736.19190112,  682.41846061, ..., 1550.46648764,
        1775.25446841,    0.        ]])

In [96]:
new_distances = distances / 60
new_distances[new_distances < 5] = 1
new_distances[new_distances >= 5] = 0
new_distances

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [140]:
population_with_prob.head()

,name,pop,prob
0,42467159,3094,0.000631
1,42467069,2863,0.000584
2,42466853,2228,0.000454
3,1413215971,3597,0.000733
4,42466495,4495,0.000917


In [101]:
from ortools.linear_solver import pywraplp

In [204]:
def min_ambu(a, offload, new_distances, population_with_prob):
    lam = population_with_prob['prob'].tolist()
    lam_K = [[0 for i in range(a)] for i in range(1163)]
    p = 88 * ((0.3 * 45 + 0.7 * (50 + offload)) + 9)/ 60 / a
    for i in range(1163):
        for k in range(a):
            lam_K[i][k] = lam[i] * ((1 - p**(k+1)) - (1 - p**k))
    solver = pywraplp.Solver('Question 1',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    objective=solver.Objective()
    objective.SetMaximization()
    x = [0 for i in range(1163)]
    y = [0 for i in range(1163)]
    y_i = [[0 for i in range(a)] for i in range(1163)]
    for i in range(1163):
        x[i] = solver.IntVar(0, solver.infinity(), 'x' + str(i))
        y[i] = solver.IntVar(0, solver.infinity(), 'y' + str(i))
        for j in range(a):
            y_i[i][j] = solver.IntVar(0, 1, 'y' + str(i) + str(j))
            objective.SetCoefficient(y_i[i][j], lam_K[i][j])
    
    total = [0]
    total[0] = solver.Constraint(a, a)
    for i in range(1163):
        total[0].SetCoefficient(x[i], 1)

    y_constraint = [0 for i in range(1163)]
    for i in range(1163):
        y_constraint[i] = solver.Constraint(0, 0)
        y_constraint[i].SetCoefficient(y[i], -1)
        for j in range(1163):
            y_constraint[i].SetCoefficient(x[j], new_distances[i][j])

    final_y_constraint = [0 for i in range(1163)]
    for i in range(1163):
        final_y_constraint[i] = solver.Constraint(0, 0)
        final_y_constraint[i].SetCoefficient(y[i], -1)
        for j in range(a):
            final_y_constraint[i].SetCoefficient(y_i[i][j], 1)
    status = solver.Solve()
    if status == solver.OPTIMAL:
        print('With total ambu equals to ' + str(a))
    elif status == solver.FEASIBLE:
        print('Solver claims feasibility but not optimality')
    else:
        print('Solver ran to completion but did not find an optimal solution')

    print('Percent is ' + str(objective.Value()))
    final_x = [0 for i in range(1163)]
    for i in range(1163):
        final_x[i] = x[i].solution_value()
    return final_x, objective.Value()

In [205]:
print("For offload equals to " + str(15))
for a in range(120,250):
    if (min_ambu(a, 15, new_distances, population_with_prob)[1] > 0.9):
        break

For offload equals to 15
With total ambu equals to 120
Percent is 0.6781282854137454
With total ambu equals to 121
Percent is 0.6940825779024113
With total ambu equals to 122
Percent is 0.7091684350417711
With total ambu equals to 123
Percent is 0.7235440762028545
With total ambu equals to 124
Percent is 0.7372193263440556
With total ambu equals to 125
Percent is 0.7501880871608203
With total ambu equals to 126
Percent is 0.7624902810746522
With total ambu equals to 127
Percent is 0.7741781149780995
With total ambu equals to 128
Percent is 0.7852594401324295
With total ambu equals to 129
Percent is 0.7957824720062223
With total ambu equals to 130
Percent is 0.8057679326772862
With total ambu equals to 131
Percent is 0.8152569096129345
With total ambu equals to 132
Percent is 0.8242484390733049
With total ambu equals to 133
Percent is 0.8327852273167734
With total ambu equals to 134
Percent is 0.840877515505345
With total ambu equals to 135
Percent is 0.8485188439090262
With total ambu 

In [206]:
print("For offload equals to " + str(30))
for a in range(140,250):
    if (min_ambu(a, 30, new_distances, population_with_prob)[1] > 0.9):
        break

For offload equals to 30
With total ambu equals to 140
Percent is 0.741419189034512
With total ambu equals to 141
Percent is 0.7539283868778524
With total ambu equals to 142
Percent is 0.7658312082606735
With total ambu equals to 143
Percent is 0.777115995543718
With total ambu equals to 144
Percent is 0.7878284503846782
With total ambu equals to 145
Percent is 0.7980114642812406
With total ambu equals to 146
Percent is 0.8076624110073243
With total ambu equals to 147
Percent is 0.8168158088701288
With total ambu equals to 148
Percent is 0.8254887930206612
With total ambu equals to 149
Percent is 0.8337246275762422
With total ambu equals to 150
Percent is 0.8415737048706649
With total ambu equals to 151
Percent is 0.8490311431498956
With total ambu equals to 152
Percent is 0.8560885750522758
With total ambu equals to 153
Percent is 0.8627813658626304
With total ambu equals to 154
Percent is 0.8691421812573291
With total ambu equals to 155
Percent is 0.8752868551338138
With total ambu e

In [207]:
print("For offload equals to " + str(45))
for a in range(160,250):
    if (min_ambu(a, 45, new_distances, population_with_prob)[1] > 0.9):
        break

For offload equals to 45
With total ambu equals to 160
Percent is 0.7909693936617583
With total ambu equals to 161
Percent is 0.8008046105620873
With total ambu equals to 162
Percent is 0.8101454202862277
With total ambu equals to 163
Percent is 0.8190199432575652
With total ambu equals to 164
Percent is 0.8274411587234709
With total ambu equals to 165
Percent is 0.8354393615637493
With total ambu equals to 166
Percent is 0.8430350417996175
With total ambu equals to 167
Percent is 0.8502472846387188
With total ambu equals to 168
Percent is 0.857094291739102
With total ambu equals to 169
Percent is 0.8635916521730135
With total ambu equals to 170
Percent is 0.86984331559982
With total ambu equals to 171
Percent is 0.875801426818138
With total ambu equals to 172
Percent is 0.8814861145735831
With total ambu equals to 173
Percent is 0.8869246278012616
With total ambu equals to 174
Percent is 0.8920904662282606
With total ambu equals to 175
Percent is 0.8969986983030391
With total ambu equ

In [208]:
print("For offload equals to " + str(60))
for a in range(180,250):
    if (min_ambu(a, 60, new_distances, population_with_prob)[1] > 0.9):
        break

For offload equals to 60
With total ambu equals to 180
Percent is 0.8297467398289075
With total ambu equals to 181
Percent is 0.837504002016428
With total ambu equals to 182
Percent is 0.8448827539326137
With total ambu equals to 183
Percent is 0.8518825002805275
With total ambu equals to 184
Percent is 0.8585408811014874
With total ambu equals to 185
Percent is 0.8648802201231109
With total ambu equals to 186
Percent is 0.8710001197214943
With total ambu equals to 187
Percent is 0.8768237758135996
With total ambu equals to 188
Percent is 0.8823964810940643
With total ambu equals to 189
Percent is 0.8876964224622876
With total ambu equals to 190
Percent is 0.8927206560260267
With total ambu equals to 191
Percent is 0.8975144903423418
With total ambu equals to 192
Percent is 0.902128465823764


Based on the results we have for offload time 30 minutes, a = 160 and a * (1 - p) = 45. Thus we starts with 45.

In [235]:
p = 88 * ((0.3 * 45 + 0.7 * (50 + 30)) + 9)/ 60 / 160
160 * (1 - p)

44.86666666666666

In [330]:
def min_ambu_for_table(a, offload, new_distances, population_with_prob, old_x, up, p):
    lam = population_with_prob['prob'].tolist()
    lam_K = [[0 for i in range(a)] for i in range(1163)]
    for i in range(1163):
        for k in range(a):
            lam_K[i][k] = lam[i] * ((1 - p**(k+1)) - (1 - p**k))
    solver = pywraplp.Solver('Question 2',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    objective=solver.Objective()
    objective.SetMaximization()
    x = [0 for i in range(1163)]
    y = [0 for i in range(1163)]
    y_i = [[0 for i in range(a)] for i in range(1163)]
    for i in range(1163):
        x[i] = solver.IntVar(0, solver.infinity(), 'x' + str(i))
        y[i] = solver.IntVar(0, solver.infinity(), 'y' + str(i))
        for j in range(a):
            y_i[i][j] = solver.IntVar(0, 1, 'y' + str(i) + str(j))
            objective.SetCoefficient(y_i[i][j], lam_K[i][j])
    
    total = [0]
    total[0] = solver.Constraint(a, a)
    for i in range(1163):
        total[0].SetCoefficient(x[i], 1)

    y_constraint = [0 for i in range(1163)]
    for i in range(1163):
        y_constraint[i] = solver.Constraint(0, 0)
        y_constraint[i].SetCoefficient(y[i], -1)
        for j in range(1163):
            y_constraint[i].SetCoefficient(x[j], new_distances[i][j])

    final_y_constraint = [0 for i in range(1163)]
    for i in range(1163):
        final_y_constraint[i] = solver.Constraint(0, 0)
        final_y_constraint[i].SetCoefficient(y[i], -1)
        for j in range(a):
            final_y_constraint[i].SetCoefficient(y_i[i][j], 1)
    
    x_constraint = [0 for i in range(1163)]
    if up == 1:
        for i in range(1163):
            solver.Add(x[i] >= old_x[i])
    elif up == 0:
        for i in range(1163):
            solver.Add(x[i] <= old_x[i])
            
    status = solver.Solve()
#     if status == solver.OPTIMAL:
#         print('With total ambu equals to ' + str(a))
#     elif status == solver.FEASIBLE:
#         print('Solver claims feasibility but not optimality')
#     else:
#         print('Solver ran to completion but did not find an optimal solution')

#     print('Percent is ' + str(objective.Value()))
    final_x = [0 for i in range(1163)]
    for i in range(1163):
        final_x[i] = x[i].solution_value()
    return final_x, objective.Value()

In [331]:
p = 88 * ((0.3 * 45 + 0.7 * (50 + 30)) + 9)/ 60 / 160
table = [[0 for i in range(1163)] for i in range(160)]
old_x = min_ambu_for_table(45, 30, new_distances, population_with_prob, [0 for i in range(1163)], -1, p)[0]
table[44] = old_x
# table[43] = min_ambu_for_table(44, 30, new_distances, population_with_prob, table[44], 0, p)[0]
# table[42] = min_ambu_for_table(43, 30, new_distances, population_with_prob, table[43], 0, p)[0]
# table[41] = min_ambu_for_table(42, 30, new_distances, population_with_prob, table[42], 0, p)[0]
# from 44 to 0
for i in range(43, -1, -1):
    new_x, objective = min_ambu_for_table(i + 1, 30, new_distances, population_with_prob, table[i + 1], 0, p)
    if objective == 0:
        table[i] = table[i + 1]
        continue
    table[i] = new_x
# from 46 to 160
for i in range(45, 160):
    new_x = min_ambu_for_table(i + 1, 30, new_distances, population_with_prob, table[i - 1], 1, p)[0]
    table[i] = new_x

In [356]:
position = []
for i in range(1163):
    if table[0][i] > 0:
        position.append(i)
        break
for j in range(159):
    for i in range(1163):
        if table[j+1][i] - table[j][i] > 0:
            position.append(i)
            break

new_position = []
for i in range(len(position)):
    new_position.append(dfp.ix[position[i]][0])
new_position

/Users/aaronyan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


[42447207,
 1770713491,
 42860516,
 42860516,
 42446036,
 42446036,
 1260335197,
 42439225,
 42439225,
 42447207,
 42439225,
 42750730,
 42437204,
 482840749,
 482840749,
 42439225,
 42446036,
 42736037,
 42446036,
 42446036,
 42432990,
 42877863,
 42437204,
 42432990,
 42447207,
 42447207,
 42447207,
 42436575,
 42447207,
 42736037,
 42732863,
 2408379355,
 42445219,
 42436489,
 42432990,
 42445219,
 42466196,
 42466196,
 42445219,
 2408379355,
 42755148,
 42429637,
 42439910,
 42439910,
 1770713491,
 42439225,
 42438798,
 42438798,
 42435981,
 42437204,
 1770713491,
 42438043,
 42458034,
 42435362,
 42435362,
 42451018,
 42452556,
 42451409,
 1858067878,
 42430770,
 42466188,
 42466188,
 42447207,
 42447207,
 42446036,
 42438498,
 42436489,
 42438798,
 42438791,
 1858067878,
 42438498,
 42451409,
 42446036,
 42452556,
 42452556,
 42451018,
 42451018,
 42466468,
 42466179,
 42466468,
 42466468,
 42447207,
 42445219,
 42447207,
 1770713491,
 42466179,
 42466179,
 42446036,
 42466179,
 